In [1]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from collections import Counter
from time import sleep
from sklearn.metrics import accuracy_score
session = requests.session()

### Чтобы найти новую часть, пролистайте ниже. Там большими буквами будет написано, не пропустите!
p.s. часть из дз1 немного отличается по двум причинам: сохранять только словарь частотных слов теперь недостаточно, поэтому еще сохранялись тесты целиком, а также возникли проблемы с блокировкой меня сайтом, но это уже не так важно.

#### Функция ниже получает код страницы для поиска тегов

In [2]:
def get_soup(link_part):
    ua = UserAgent(verify_ssl=False)
    headers = {'User-Agent': ua.random}
    link = 'https://irecommend.ru' + link_part
    response = session.get(link, headers=headers)
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    return soup

#### Функция для выбора слов и добавления их в словарь
Берутся тексты длиной более 200 слов. 

Сначала я составляла обычные каунтеры, но тогда результат получался плохой - получалось очень много названий конкретных фильмов и имен. Я связываю это с тем, что некоторые отзывы были значительно больше остальных, поэтому слова из этих отзывов значительно перевешивали. Чтобы попробовать от этого избавиться, я считаю частотность по тексту (кол-во этот слова/кол-во всех слов в тексте) и ее прибавляю к текущему значению в словаре. 

In [3]:
def get_text_lemmas(text, freq, count):
    counter = Counter()
    text = text.text
    words = nltk.word_tokenize(text.lower())
    lemmas = [morph.parse(word)[0].normal_form for word in words]
    if len(lemmas) > 200:
        count += 1
        for lemma in lemmas:
            if lemma.isalpha():
                counter[lemma] += 1
    for word in dict(counter).keys():
        freq[word] += counter[word]/len(lemmas)
    return freq, count

#### Функция собирает не менее 30 отзывов и собирает словарь
По каждому фильму не больше 2 плохих и 2 хороших отзывов, чтобы не было перевеса слов, относящихся к определенному жанру или фильму.

In [ ]:
b_list = []
g_list = []
b_freq = Counter()
g_freq = Counter()

In [114]:
def find_words(link):
    b_count_all = 0
    g_count_all = 0
    global b_list
    global g_list
    global b_freq
    global g_freq

    
    soup = get_soup(link)
    all_films = soup.find_all('a', {'class': 'read-all-reviews-link-bottom read-all-reviews-link'})
    
    for film in all_films:
        b_count = 0
        g_count = 0
        if b_count_all > 30 and g_count_all > 30:
            break
        sleep(10)
        print(film.get('href'))
        soup = get_soup(film.get('href'))
        all_reviews = soup.find_all('a', {'class': 'more'})
        
        for review in all_reviews:
            sleep(10)
            soup = get_soup(review.get('href'))
            recommend = soup.find('span', {'class': 'verdict'})
            
            if recommend.text == 'не рекомендует' and b_count <= 2 and b_count_all <= 30:
                text = soup.find('div', {'class': 'description hasinlineimage'})
                b_list.append(text.text)
                b_freq, b_count = get_text_lemmas(text, b_freq, b_count)
                b_count_all += b_count
                
            elif g_count <= 2 and g_count_all <= 30:
                text = soup.find('div', {'class': 'description hasinlineimage'})
                g_list.append(text.text)
                g_freq, g_count = get_text_lemmas(text, g_freq, g_count)
                g_count_all += g_count
                
            elif g_count > 2 and b_count > 2:
                break
                
    return b_freq, g_freq, b_count_all, g_count_all, g_list, b_list

#### Собираем словари и смотрим содержимое
Было собрано по 33 отзыва каждой категории. Очень много частотных слов, списки ни о чем не говорят.

In [124]:
link = '/category/filmy?page=1'
bad_freq, good_freq, b_count_all, g_count_all, g_list, b_list = find_words(link)

/content/unesennye-vetrom-gone-wind
/content/belosnezhka-skazka-dlya-vzroslykh-blanche-comme-neige
/content/gattaka
/content/neuyazvimyi-unbreakable
/content/strakh-glubiny-deep


In [123]:
g_freq1 = good_freq
b_freq1 = bad_freq
g_list1 = g_list
b_list1 = b_list

In [128]:
g_list.extend(g_list1)
b_list.extend(b_list1)

In [152]:
def join_freq(freq1, freq2):
    for f in freq1.keys():
        if f in freq2.keys():
            freq2[f] += freq1[f]
        else:
            freq2[f] = freq1[f]
    return freq2

In [155]:
full_g_freq = join_freq(good_freq, g_freq1)
full_b_freq = join_freq(bad_freq, b_freq1)

In [236]:
new_bad_freq = select_words(full_b_freq)
new_good_freq = select_words(full_g_freq)
pos_freq, neg_freq = delete_common_words(new_good_freq, new_bad_freq)
adj_neg = get_adj(neg_freq)
adj_pos = get_adj(pos_freq)

In [237]:
adj_neg.most_common(15)

[('непонятный', 0.012634654589946833),
 ('неприятный', 0.012463869882587325),
 ('ходячий', 0.009478672985781991),
 ('смешной', 0.008785445722234839),
 ('банальный', 0.007743268876872116),
 ('гадкий', 0.007679180887372013),
 ('смазливый', 0.007380073800738007),
 ('средний', 0.007344415957672649),
 ('никакой', 0.007079809476715317),
 ('животное', 0.006947607489814127),
 ('конкретный', 0.006674731748369451),
 ('голый', 0.006560572947127568),
 ('следующий', 0.005972816336137845),
 ('китайский', 0.005641041486425246),
 ('противный', 0.005119453924914676)]

In [240]:
adj_pos.most_common(15)

[('неуязвимый', 0.12949969358042082),
 ('идеальный', 0.043395355387284916),
 ('приятный', 0.030959912258421974),
 ('нормальный', 0.03042915279319275),
 ('страшный', 0.029386409334314995),
 ('научный', 0.029156259521878135),
 ('золотой', 0.028009698783410704),
 ('разумный', 0.022236655462407832),
 ('тёмный', 0.02222037168451984),
 ('местный', 0.021839620781679177),
 ('естественный', 0.021418031260297128),
 ('невероятный', 0.02080485896478325),
 ('книжный', 0.020753336014244415),
 ('опасный', 0.0191455839241667),
 ('генный', 0.01892147587511826)]

#### Функция удаляет стоп-слова
Я не стала отбирать слова по частотности, потому что не ясно, где устанавливать нижнюю границу.

In [230]:
def select_words(freq):
    stopWords = set(stopwords.words('russian'))
    selected_freq = Counter()
    for word in dict(freq).keys():
        if word not in stopWords:
            selected_freq[word] = freq[word]
    return selected_freq

#### Функция удаляет общие для двух словарей слова

In [223]:
def delete_common_words(good_freq, bad_freq):
    good = dict(good_freq)
    bad = dict(bad_freq)
    keys = list(bad.keys())
    for b_key in keys:
        if b_key in good.keys():
            del bad[b_key]
            del good[b_key]
    return Counter(good), Counter(bad)

In [290]:
# это мы уже сделали 

# new_bad_freq = select_words(bad_freq)
# new_good_freq = select_words(good_freq)
# pos_freq, neg_freq = delete_common_words(new_good_freq, new_bad_freq)

In [288]:
# pos_freq.most_common(15)

Как видно, в списках очень много каких-то случайных существительных. Я решила, что будет намного показательнее, если в списках останутся только прилагательные. Это должно увеличить точность оценки, даже несмотря на то, что некоторые "полезные" существительные и глаголы уберутся.

#### Функция для отбора прилагательных

In [235]:
def get_adj(freq):
    result = Counter()
    for word in dict(freq).keys():
        if morph.parse(word)[0].tag.POS == 'ADJF':
            result[word] = freq[word]
    return result

In [289]:
# это тоже сделали 

# adj_neg = get_adj(neg_freq)
# adj_pos = get_adj(pos_freq)

In [291]:
# adj_neg.most_common(15)

In [292]:
# adj_pos.most_common(15)

Получились относительно нормальные списки с наличием подходящих оценочных слов.

#### Функция собирает тестовую выборку
Скачивается не менее 10 отзывов с их оценкой.

In [241]:
# test_link = 'https://irecommend.ru/category/filmy?page=499'

def get_test_texts(n):
    text_dict = {}
    b_count_all = 0
    g_count_all = 0

    soup = get_soup('/category/filmy?page=499')
    all_books = soup.find_all('a', {'class': 'read-all-reviews-link-bottom read-all-reviews-link'})

    for book in all_books:
        b_count = 0
        g_count = 0
        if b_count_all > n and g_count_all > n:
            break
        sleep(10)
        soup = get_soup(book.get('href'))
        all_reviews = soup.find_all('a', {'class': 'more'})
        
        for review in all_reviews:
            sleep(10)
            soup = get_soup(review.get('href'))
            recommend = soup.find('span', {'class': 'verdict'})
            
            if recommend.text == 'не рекомендует' and b_count <= 2 and b_count_all <= n:
                text = soup.find('div', {'class': 'description hasinlineimage'})
                text_dict[text.text] = recommend.text
                b_count += 1
                b_count_all += b_count
                
            elif g_count <= 2 and g_count_all <= n:
                text = soup.find('div', {'class': 'description hasinlineimage'})
                text_dict[text.text] = recommend.text
                g_count += 1
                g_count_all += g_count
                
            elif g_count > 2 and b_count > 2:
                break
                
    return text_dict

In [242]:
test_dict = get_test_texts(10)

# Новая часть! ДЗ2!

В первой части дз2 Рymorphy покзал лучшие результаты, поэтому беру его.

In [16]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from pymorphy2.tokenizers import simple_word_tokenize

#### Функция составляет словарь с биграммами вида "не" + любое слово
Частица "не" имеет особенно важное значение для определения негативных отзывов, поэтому я взяла такой биграмм.

In [168]:
def find_neg_pair(pos_tagged, neg_pairs):
    counter = Counter()
    for i in range(len(pos_tagged)-1):
        if list(pos_tagged[i].keys())[0].lower() == 'не':
            pair = 'не ' + list(pos_tagged[i+1].keys())[0].lower()
            counter[pair] += 1
    
    for word in dict(counter).keys():
        neg_pairs[word] += counter[word]/len(pos_tagged)
    return neg_pairs

#### Функция составляет словарь с биграммами из двух заданных частей речи
Я выбрала следующие биграммы:
* Прилагательное + существительное - прилагательные часто используются именно для оценки предметов и понятий, поэтому такой биграмм может быть показательным
* Глагол + существительное - это должна быть модель глагола и его объекта, которая покажет пары типа "понравилась идея", "оценила сюжет" и т.п.
* Существительное + глагол - эта модель должна включать оценки того, как фильм повлиял на эмоции человека: "фильм впечатлил", "сюжет разочаровал" и т.п.

Эксперимернтальным методом выяснилось, что последние два типа биграммов не улучшают качество, а наоборот ухудшают, поэтому они не будут добавлены...

In [167]:
def find_pos_pair(pos_tagged, pos1, pos2, pairs):
    counter = Counter()
    for i in range(len(pos_tagged)-1):
        if list(pos_tagged[i].values())[0] == pos1 and list(pos_tagged[i+1].values())[0] == pos2:
            pair = list(pos_tagged[i].keys())[0].lower() + ' ' + list(pos_tagged[i+1].keys())[0].lower()
            counter[pair] += 1
            
    for word in dict(counter).keys():
        pairs[word] += counter[word]/len(pos_tagged)
    return pairs

#### Функция находит все нужные мне биграммы в тексте

In [313]:
def find_ngrams(text_list):
    neg_pairs = Counter()
    obj = Counter()
    subj = Counter()
    adj_noun = Counter()
    for text in text_list:
        pos_tagged = []
        tokens = simple_word_tokenize(text.lower())
        lemmas = [morph.parse(word)[0].normal_form for word in tokens]

        for word in lemmas:
            if word[0].isalpha():
                pos_tagged.append({word:str(morph.parse(word)[0].tag.POS)})
        
        neg_pairs = find_neg_pair(pos_tagged, neg_pairs)
        obj = find_pos_pair(pos_tagged, 'INFN', 'NOUN', obj)
        subj = find_pos_pair(pos_tagged, 'NOUN', 'INFN', subj)
        adj_noun = find_pos_pair(pos_tagged, 'ADJF', 'NOUN', adj_noun)
        
    return neg_pairs, obj, subj, adj_noun
        

In [314]:
b_neg, b_obj, b_subj, b_adj_noun = find_ngrams(b_list)
g_neg, g_obj, g_subj, g_adj_noun = find_ngrams(g_list)

In [315]:
bad_pairs = b_neg
# bad_pairs.update(b_obj)
# bad_pairs.update(b_subj)
bad_pairs.update(b_adj_noun)

In [316]:
good_pairs = g_neg
# good_pairs.update(g_obj)
# good_pairs.update(g_subj)
good_pairs.update(g_adj_noun)

In [317]:
new_bad_pairs = select_pairs(bad_pairs)
new_good_pairs = select_pairs(good_pairs)
pos_pairs, neg_pairs = delete_common_words(new_good_pairs, new_bad_pairs)

In [318]:
pos_pairs.most_common(15)

[('звёздный война', 0.0323936690811007),
 ('не пожалеть', 0.024574369690326118),
 ('весь часть', 0.02247191011235955),
 ('тёмный сторона', 0.02222321855571497),
 ('приятный просмотр', 0.018682506204139752),
 ('эротический мелодрама', 0.01680672268907563),
 ('семейный пара', 0.015411319166841995),
 ('замечательный фильм', 0.01242785016940981),
 ('хороший часть', 0.011806551889048302),
 ('реальный событие', 0.011366557905013785),
 ('не решаться', 0.011235955056179775),
 ('не откладывать', 0.011235955056179775),
 ('прекрасный момент', 0.011235955056179775),
 ('интересный герой', 0.011235955056179775),
 ('весь сага', 0.010741047358254263)]

In [319]:
neg_pairs.most_common(15)

[('не рекомендовать', 0.026928894626102626),
 ('положительный отзыв', 0.014616429457272782),
 ('не вдохновить', 0.012195121951219513),
 ('не глупый', 0.012195121951219513),
 ('хвалебный речь', 0.012195121951219513),
 ('внутренний дрожь', 0.012195121951219513),
 ('глупый кокетка', 0.012195121951219513),
 ('сильный человек', 0.012195121951219513),
 ('сильный книга', 0.012195121951219513),
 ('не понравиться', 0.011920233243866598),
 ('не притягивать', 0.008403361344537815),
 ('несгибаемый характер', 0.008403361344537815),
 ('безответственный эгоистка', 0.008403361344537815),
 ('каждый минута', 0.008403361344537815),
 ('мировой шедевр', 0.008403361344537815)]

#### Функция отбирает нужные биграммы
Убираются биграммы, где одно слово является стопсловом (не считаем "не"), чтобы убрать бесполезные биграммы вроде "не так". Еще устанавливается нижняя граница частотности

In [263]:
def select_pairs(freq):
    stopWords = set(stopwords.words('russian'))
    stopWords.remove('не')
    selected_freq = Counter()
    for word in dict(freq).keys():
        if word.split()[0] not in stopWords and word.split()[1] not in stopWords and freq[word] > 0.001:
            selected_freq[word] = freq[word]
    return selected_freq

### Тут еще кусочек старой домашки - оценка результатов
Но это нужно посмотреть, чтобы сравнить результаты.

#### Функция, предсказывающая оценку
Итоговые положительные и отрицательные очки делятся на суммы всех значений в соответствующих словарях, чтобы получить процентные соотношения. Нельзя сравнивать очки, потому что суммы значений в разных оценках могут отличаться.

In [245]:
def predict_verdict(text, adj_pos, adj_neg):
    pos_score = 0
    neg_score = 0
    words = nltk.word_tokenize(text.lower())
    lemmas = [morph.parse(word)[0].normal_form for word in words]
    for lemma in lemmas:
        if lemma in adj_pos.keys():
            pos_score += adj_pos[lemma]
        elif lemma in adj_neg.keys():
            neg_score += adj_neg[lemma]
    n_score = neg_score/sum(adj_neg.values())
    p_score = pos_score/sum(adj_pos.values())
    return p_score, n_score

Посмотрим на совпадения предсказаний и правильных ответов. Так получилось, что в 3 текстах вообще не встретились слова из собранных словарей, поэтому предсказать оценку невозможно. 

In [293]:
# for text in test_dict.keys():
#     p_score, n_score = predict_verdict(text, adj_pos, adj_neg)
#     if p_score > n_score:
#         print('predict: positive', p_score, n_score)
#     elif p_score == n_score:
#         print('predict: can not predict')
#     else:
#         print('predict: negative', p_score, n_score)
#     print('answer:', test_dict[text], '\n')

Посчитаем accuracy. Получилось 77%. В прошлый раз получилось 75%, но сейчас набор текстов новый, поэтому есть небольшое отличие.

In [249]:
# 1 - pos, 0 - neg
predict = []
gold = []
for text in test_dict.keys():
    p_score, n_score = predict_verdict(text, adj_pos, adj_neg)
    if p_score > n_score:
        predict.append(1)
    else:
        predict.append(0)
    if test_dict[text] == 'рекомендует':
        gold.append(1)
    else:
        gold.append(0)

print("Accuracy: %.4f" % accuracy_score(predict, gold))

Accuracy: 0.7692


#### Функция предсказания оценки, принимающая во внимание собранные биграммы

In [283]:
def upd_predict_verdict(text, adj_pos, adj_neg, pos_pairs, neg_pairs):
    pos_score = 0
    neg_score = 0
    pos_score_p = 0
    neg_score_p = 0
    words = nltk.word_tokenize(text.lower())
    lemmas = [morph.parse(word)[0].normal_form for word in words]
    for lemma in lemmas:
        if lemma in adj_pos.keys():
            pos_score += adj_pos[lemma]
        elif lemma in adj_neg.keys():
            neg_score += adj_neg[lemma]
    for i in range(len(lemmas)-1):
        if lemmas[i] + ' ' + lemmas[i+1] in pos_pairs.keys():
            pos_score_p += pos_pairs[lemmas[i] + ' ' + lemmas[i+1]]
        elif lemmas[i] + ' ' + lemmas[i+1] in neg_pairs.keys():
            neg_score_p += neg_pairs[lemmas[i] + ' ' + lemmas[i+1]]
    n_score = neg_score/sum(adj_neg.values()) + neg_score_p/sum(neg_pairs.values())
    p_score = pos_score/sum(adj_pos.values()) + pos_score_p/sum(pos_pairs.values())
    return p_score, n_score

In [284]:
predict = []
gold = []
for text in test_dict.keys():
    p_score, n_score = upd_predict_verdict(text, adj_pos, adj_neg, pos_pairs, neg_pairs)
    if p_score > n_score:
        predict.append(1)
    else:
        predict.append(0)
    if test_dict[text] == 'рекомендует':
        gold.append(1)
    else:
        gold.append(0)

print("Accuracy: %.4f" % accuracy_score(predict, gold))

Accuracy: 0.9231


Получилось 92%. Качество улучшилось!